In [ ]:
import pandas as pd
import os
import math
import numpy as np

In [ ]:
# Set the working directory to 'this.dir()'
os.chdir('C:/Users/6997619/Documents/GitHub projects/DHZW-simulation_Sim-2APL/src/main/resources/DHZW_full')

# Read synthetic activities
df = pd.read_csv("DHZW_activities_locations.csv")

In [ ]:
len(df)
df

In [ ]:
# Transfrom the activity type in upper case like ODiN
df['activity_type'] = df['activity_type'].str.upper()

In [ ]:
# Sort the dataframe by 'Agent' and 'Day'
df = df.sort_values(['pid', 'day_of_week', 'activity_number']).reset_index(drop=True)

In [ ]:
# Within each agent and day, add the previous activity and postcode visited
df['previous_activity'] = df.groupby(['pid', 'day_of_week'])['activity_type'].shift()
df['previous_postcode'] = df.groupby(['pid', 'day_of_week'])['postcode'].shift()

In [ ]:
# Extract trips by selecting only activities with a previous activity within the day
df = df.dropna(subset=['previous_activity'])
len(df)

In [ ]:
# Select columns
df = df[['pid', 'day_of_week', 'previous_activity', 'activity_type', 'previous_postcode', 'postcode']]

In [ ]:
# Transfrom PC5 to PC4

df['previous_postcode'] = df['previous_postcode'].apply(lambda x: x[:-1] if len(x) == 5 else x)
print(df['previous_postcode'].str.len().unique())

df['postcode'] = df['postcode'].apply(lambda x: x[:-1] if len(x) == 5 else x)
print(df['postcode'].str.len().unique())

### Add postcode home from synthetic population

In [ ]:
# Read synthetic population to add home postcode
df_synthetic_population = pd.read_csv("DHZW_synthetic_population.csv")

# select useful coloumns
df_synthetic_population = df_synthetic_population[['pid', 'PC6']]

# Transfrom home PC6 to PC4
df_synthetic_population['PC6'] = df_synthetic_population['PC6'].str[:-2]

# Rename coloumn to match ODiN
df_synthetic_population = df_synthetic_population.rename(columns={'PC6': 'hh_PC4'})

In [ ]:
# Add home PC4 to the trips
df = pd.merge(df, df_synthetic_population, on='pid')

In [ ]:
# Rename like ODiN
df = df.rename(columns={'pid': 'agent_ID',
                        'day_of_week': 'day_of_week',
                        'activity_type': 'disp_activity',
                        'postcode': 'disp_arrival_PC4',
                        'previous_postcode': 'disp_start_PC4'})

In [ ]:
len(df)

In [ ]:
# Save trips
os.chdir('C:/Users/6997619/Documents/GitHub projects/DHZW_assign_location/data/processed/analysis/ODiN')
df.to_csv('synthetic_trips.csv', index=False, header=True)